In [31]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [32]:
# titanic dataset 을 tensor로 저장하여 관리할 수 있는 class이다.
class TitanicDataset(Dataset):
# 생성자로 매개변수 X 와 y를 각각 tensor로 저장하는 생성자이다.  
  def __init__(self, X, y):
    # float형으로 32비트를 가진다.
    self.X = torch.FloatTensor(X)
    # int형으로 64비트를 가진다.
    self.y = torch.LongTensor(y)
# len(클래스) 입력시 반환값으로 self.x tensor의 하위 텐서 개수를 출력하는 함수이다.
  def __len__(self):
    return len(self.X)
# 클래스[i] 를 입력하면 해당 위치에 저장되어있는 self.X[i] 와 self.y[i]의 값을 반환한다.
  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}
# print(클래스)를 실행하면 아래의 str 값이 반환되어 출력된다.
  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

In [33]:
# titanic test dataset이다.
class TitanicTestDataset(Dataset):
# 생성자이다. float tensor를 생성하며 크기는 32이다.
  def __init__(self, X):
    self.X = torch.FloatTensor(X)
# len(클래스) 시 len(self.X) 가 출력된다.
  def __len__(self):
    return len(self.X)
# 클래스[i] 시 self.x[i]기 출력된다.
  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}
# print(클래스)시 str이 출력됨.
  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

In [34]:
# 전처리된 dataset을 가져오는 함수이다.
def get_preprocessed_dataset():
    # os모듈을 통해 현재 파일의 절대 경로를 CURRENT_FILE_PATH에 저장한다.
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(" "))
    # 훈련 데이터 주소와 테스트 데이터 주소를 저장
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
    # 훈련 데이터와 테스트 데이터를 CSV 형식으로 읽어서 DATA FRAME으로 저장한다.
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    # 두 DATA FRAME을 연결하는 concat 함수를 이용하여 정렬하지 않으며 연결한다.
    all_df = pd.concat([train_df, test_df], sort=False)
    # 전처리함수1
    # fare 결측치를 클래스에 따른 평균값으로 추가
    all_df = get_preprocessed_dataset_1(all_df)
    # 전처리함수2
    # Nmae 값을 성, 존칭, 이름으로 분할하고 추가
    all_df = get_preprocessed_dataset_2(all_df)
    # 전처리함수3
    # 존칭에 따른 나이 평균값으로 결측치 채우기
    all_df = get_preprocessed_dataset_3(all_df)
    # 전처리함수4
    # 가족 수 feature 추가, 필요없는 [탑승자 id, 이름, 가족이름, 이름, 티켓번호, 방호수] feature 삭제
    all_df = get_preprocessed_dataset_4(all_df)
    # 전처리함수5
    # 존칭이 없다면 other로 기입, 탑승지 결측치를 missing으로 기입
    all_df = get_preprocessed_dataset_5(all_df)
    # 전처리함수6
    # object 자료형인 
    all_df = get_preprocessed_dataset_6(all_df)
    # Survived feature가 null이 아닌 요소에 대해서 열을 제거하고 리스트를 리셋하고 이전 인덱스를 버린다.
    # 결과적으로 생존 값이 있는 survived 열이 삭제된다.
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    # survived 값을 추가한 target 값을 저장한다.
    train_y = train_df["Survived"]

    # 생존이 null인 요소에 대해서 열을 제거하고 리스트 리셋 후 이전 인덱스를 버린다.
    # 결과적으로 생존 값이 없는 survived 열이 삭제된다.
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # 훈련데이터 x와 타겟데이터 y 에 대해 클래스를 선언
    dataset = TitanicDataset(train_X.values, train_y.values)
    print(dataset)
    # 8대 2의 비율로 훈련 데이터와 값 데이터를 랜덤으로 분할하여 데이터셋 생성.
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    # 테스트용 타이타닉 데이터셋을 생성
    test_dataset = TitanicTestDataset(test_X.values)
    print(test_dataset)

    # 훈련 데이터셋, 검증 데이터셋, 테스트 데이터셋 반환
    return train_dataset, validation_dataset, test_dataset

In [35]:
# 전처리함수1
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    
    # 결측치의 이유를 알면 결측치를 다른 값으로 채워 넣을 수 있다.
    # pclass을 기준으로 묶는다. 1클래스 2클래스 3클래스
    # 그룹화된 데이터에서 각 데이터를 평균을 낸다. 클래스는 같은 값으로 묶였으니 같은 값이 나오고 fare만 평균값으로 나오게 된다.
    # 그룹화된 결과를 데이터프레임 형태로 유지하며 인덱스를 처리한다.
    # 결과적으로 fare_mean 데이터는 각 클래스에 따른 fare 평균 값을 가진 데이터 프레임이 된다.
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    # Fare_mean 값의 열 이름을 변경한다.
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    # all_df에 대해서 fare_mean 데이터를 pclass에 따라 왼쪽 합병한다.
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    # all_df에 fare값이 null이 열에 대해서 fare feature 에 fare_mean 값을 넣는다.
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]
    
    return all_df

In [36]:
# 전처리함수2
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침

    # all_df에 Name feature 에 대해 ,. 문자로 분할하되 최대 2번 분할하며 확장하여 저장한다.
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    # 성, 존칭, 이름으로 열 이름 정정
    name_df.columns = ["family_name", "honorific", "name"]
    # 세 이름들에 대해서 양 옆 공백을 제거
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    # 세 data frame에 대해 열로 결합하기
    all_df = pd.concat([all_df, name_df], axis=1)
    # 전처리 함수 1의 merge와 다른 점은 merge는 해당 값에 대해서 관계가 있는 경우 쓰이고
    # 전처리 함수 2의 concat은 단순히 이어붙일때 쓰인다.
    return all_df

In [37]:
# 전처리함수3
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    
    # 존칭과 나이에 대해서 존칭으로 값을 묶고 중앙값을 계산하고 반올림하여 나이 feature을 초기화합니다.
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    # 열의 이름을 변경합니다.
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    # 존칭을 기준으로 존칭 평균 값을 왼쪽 정렬한다.
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    # 나이 feature가 null인 값에 대해서 age feature의 위치에 존칭 중앙값을 추가한다.
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    # 존칭 중앙값 feature을 제거한다.
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

### 전처리 함수 1과 3은 둘 다 새로운 feature을 생성하고 붙이는데 왜 1번은 drop하지 않고 3번만 drop하는가?

In [38]:
# 전처리함수4
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    # 가족 수 collum을 추가하고 동승 부모/자식, 동승 자매/배우자 를 더하여 저장
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
    
    # 혼자탑승(alone) 컬럼 새롭게 추가
    # 동승자가 없다면 alone feature 을 추가하여 1를 저장
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    # alone feature에 누락된 값에 대해 0을 기입한다.
    # inplace 는 반환값을 없애고 원래 데이터프레임에 정정한다.
    # 이런 사소한 코드로 사이즈가 큰 데이터의 처리를 진행하자.
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    # 탑승자 id, 이름, 가족이름, 이름, 티켓번호, 방호수
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

In [39]:
# 전처리함수5
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    # 존칭이 있는 것이 아니라면 other로 기입
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    # 탑승지에 대해서 빈칸에 대해서 missing을 기입
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

In [40]:
# 전처리함수6
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    # data frame에서 타입이 object인 feature에 대해서 category_feature dataframe에 저장한다.
    category_features = all_df.columns[all_df.dtypes == "object"]
    # LableEncoder 모듈을 가져와서 범주형 데이터를 숫자로 변환하는데 쓰인다고 함.
    from sklearn.preprocessing import LabelEncoder
    # category_features의 모든 열에 대해서
    for category_feature in category_features:
        # 클래스 객체 생성
        le = LabelEncoder()
        # 데이터 프레임의 타입이 객체라면 
        if all_df[category_feature].dtypes == "object":
          # 해당 데이터 프레임에 대해서 학습하여 고유값에 숫자를 할당
          le = le.fit(all_df[category_feature])
          # 변환된 결과를 원래 데이터프레임에 적용, 각 고유한 값이 해당하는 숫자로 대체
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

In [41]:
## nn 모듈을 이용해서 nn.Module클래스 상속받아서 클래스 선언
from torch import nn
class MyModel(nn.Module):
    # 상속 클래스의 생성자 받기
  def __init__(self, n_input, n_output):
    super().__init__()
# 추가 생성자 속성으로 아래 추가
      # 내 모듈은 아래의 순서를 통해 신경망 모델 생성 및 진행됨
    self.model = nn.Sequential(
        # 입력, 30개의 neuron을 가진 layer 생성
      nn.Linear(n_input, 30),
        # ReLU 과정 거침
      nn.ReLU(),
        # 30개, 30개 neuron을 가진 Layer 생성
      nn.Linear(30, 30),
        # ReLU 과정 거침
      nn.ReLU(),
        # 30개, 출력 neuron을 가진 Layer 생성
      nn.Linear(30, n_output),
    )
      
  def forward(self, x):
    x = self.model(x)
    return x

In [42]:
# 테스트 데이터를 로더하여 테스트 함수 실행
def test(test_data_loader):
  print("[TEST]")
  # 테스트 데이터 로더에서 첫 미니 배치를 가져온다
  batch = next(iter(test_data_loader))
  print(batch)
  # 배치의 사이즈를 출력한다.
  print("{0}".format(batch['input'].shape))
  # 클래스로 입력이 11이고 출력이 2인 클래스 선언
  my_model = MyModel(n_input=11, n_output=2)
  # 내 클래스에 배치 입력
  output_batch = my_model(batch['input'])
  # 예측 배치
  prediction_batch = torch.argmax(output_batch, dim=1)
  # 예측값 출력
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())

In [43]:
if __name__ == "__main__":
# 전처리 데이터셋을 가져와서 훈련, 검증, 테스트 데이터셋 선언
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
# 각각의 데이터 셋의 개수 출력
    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))

    print("#" * 50, 1)
# 훈련 데이터셋에 대해서 넘버링, 입력값, 타겟값 출력
    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)
#훈련 데이터 로더 - 데이터 셋(훈련 데이터), 배치 크기 16, 섞어서 가져오기
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
#검증 데이터 로더 - 데이터 셋(검증 데이터), 배치 크기 16, 섞어서 가져오기
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
#테스트 데이터 로더 - 데이터 셋(테스트 데이터), 배치 크기 테스트 데이터 셋의 개수 , 섞어서 가져오기
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    print("[TRAIN]")
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("[VALIDATION]")
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("#" * 50, 3)

    test(test_data_loader)

Data Size: 891, Input Shape: torch.Size([891, 11]), Target Shape: torch.Size([891])
Data Size: 418, Input Shape: torch.Size([418, 11])
train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  1.0000, 29.0000,  8.0000,  2.0000, 69.5500,  2.0000, 13.3029,
         2.0000, 10.0000,  0.0000]): 0
1 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  6.9500,  1.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([  1.0000,   1.0000,  29.0000,   0.0000,   0.0000, 227.5250,   0.0000,
         87.5090,   2.0000,   0.0000,   1.0000]): 0
3 - tensor([  1.0000,   0.0000,  21.0000,   2.0000,   2.0000, 262.3750,   0.0000,
         87.5090,   1.0000,   4.0000,   0.0000]): 1
4 - tensor([ 2.0000,  1.0000, 29.0000,  0.0000,  0.0000,  0.0000,  2.0000, 21.1792,
         2.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  1.0000, 26.0000,  1.0000,  0.0000,  7.8542,  2.0000, 13.3029,
         2.0000,  1.00